<a href="https://colab.research.google.com/github/hangatzu2017/RAG/blob/main/RAG_MR_LOTR_RERANK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In questo notebook vedremo 3 miglioramenti chiave per i sistemi R.A.G. (Retrival Augmented Generation)

1. Il primo problema è rappresentato dall'avere diversi tipi di documenti, in questo caso infatti, è impossibile avere un unico indice. È necessario quindi creare più indici e indicizzarli tutti separatamente per eseguire la R.A.G.. Vedremo come migliorare i risultati in questo caso utilizzando Merger Retriever.

2. Lost in the Middle! Indipendentemente dall'architettura del modello, quando si includono più di 10 documenti recuperati le prestazioni si riducono notevolmente. In breve: Quando i modelli devono accedere a informazioni rilevanti all'interno di lunghi contesti, tendono a ignorare i documenti forniti. Vedi: https://arxiv.org/abs/2307.03172
Per evitare questo problema, è possibile riordinare i documenti dopo il recupero per evitare un calo delle prestazioni. Questa tecninca prende il nome di Long Context Reorder.

3. Spesso l'ordine (score) con cui i chnk dei documenti indicizzati vengono recuperati non è esattamente quello voluto e le informazioni più pertinenti rischiano di finire nel dimenticatoio, fortunatamente esiste una tecnica chiamata Rerank che promette di farci dimenticare questo genere di problema.


## 1. Prerequisiti e Librerie

In [41]:
!pip install -U langchain sentence_transformers pypdf chromadb flashrank --no-cache

In [42]:
import os, yaml
from langchain.vectorstores import Chroma
from langchain.chat_models import AzureChatOpenAI
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.document_loaders import PyPDFLoader, TextLoader
from langchain.retrievers.merger_retriever import MergerRetriever
from langchain.document_transformers import (
                                            EmbeddingsRedundantFilter,
                                            EmbeddingsClusteringFilter,
                                            LongContextReorder
                                            )
from langchain.retrievers.document_compressors import DocumentCompressorPipeline
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.retrievers import ContextualCompressionRetriever

## 2. Definizione il modello di embedding da utilizzare

In [43]:
print("Caricamento del modello di Embeddings...")
embedding = HuggingFaceBgeEmbeddings(
                                    model_name="intfloat/multilingual-e5-base",
                                    model_kwargs={'device': 'cuda'},
                                    encode_kwargs={'normalize_embeddings': True}
                                    )



Caricamento del modello di Embeddings...


## 3. Preprocessamento dei Dati

In [44]:
loader_un_sdg = PyPDFLoader("https://www.maristi.it/ciao/wp-content/uploads/2020/02/scuolaguida-manuale-teoria-A1-A-B.pdf")
documents_un_sdg = loader_un_sdg.load()
text_splitter_un_sdg = RecursiveCharacterTextSplitter(
                                                    chunk_size=1000,
                                                    chunk_overlap=100
                                                    )
texts_un_sdg = text_splitter_un_sdg.split_documents(documents_un_sdg)

In [45]:
texts_un_sdg[0]

Document(page_content='Valerio Platia \nRoberto Mastri \n \n \n \nManuale di  teoria \nper le patenti A1, A e  B\nEdizione 2008', metadata={'source': 'https://www.maristi.it/ciao/wp-content/uploads/2020/02/scuolaguida-manuale-teoria-A1-A-B.pdf', 'page': 0})

In [46]:
loader_second_doc = PyPDFLoader("https://fad.unich.it/pluginfile.php/37229/course/section/1893/IT%20concetti%20Base%201.pdf")
documents_second = loader_second_doc.load()
text_splitter_doc_second = RecursiveCharacterTextSplitter(
                                                                chunk_size=1000,
                                                                chunk_overlap=100
                                                                )
texts_second_doc = text_splitter_doc_second.split_documents(documents_second)

In [47]:
texts_second_doc[0]

Document(page_content="Corso di Informatica\nI Concetti base dell'informatica", metadata={'source': 'https://fad.unich.it/pluginfile.php/37229/course/section/1893/IT%20concetti%20Base%201.pdf', 'page': 0})

## 4.Vector Store

In [48]:
if not os.path.exists("db/18/un_sdg_chroma_cosine"):
    un_sdg_store = Chroma.from_documents(
                                        texts_un_sdg,
                                        embedding,
                                        collection_metadata={"hnsw:space": "cosine"},
                                        persist_directory="db/18/un_sdg_chroma_cosine"
                                        )
else:
    un_sdg_store = Chroma(
                        persist_directory="db/18/un_sdg_chroma_cosine",
                        embedding_function=embedding
                        )

if not os.path.exists("db/18/second_chroma_cosine"):
    second_doc_store = Chroma.from_documents(
                                        texts_second_doc,
                                        embedding,
                                        collection_metadata={"hnsw:space": "cosine"},
                                        persist_directory="db/18/second_chroma_cosine"
                                        )
else:
    second_doc_store = Chroma(
                        persist_directory="db/18/second_chroma_cosine",
                        embedding_function=embedding
                        )

## 5. Costruire il Merged Retriever

In [49]:
retriever_un_sdg = un_sdg_store.as_retriever(
                                            search_type = "similarity",
                                            search_kwargs = {
                                                            "k":3,
                                                            "include_metadata": True
                                                            }
                                            )

retriever_second_doc = second_doc_store.as_retriever(
                                                                search_type = "similarity",
                                                                search_kwargs = {
                                                                                "k":3,
                                                                                "include_metadata": True
                                                                                }
                                            )
lotr = MergerRetriever(retrievers=[retriever_un_sdg, retriever_second_doc])

lotr

MergerRetriever(retrievers=[VectorStoreRetriever(tags=['Chroma', 'HuggingFaceBgeEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x7e3dd38e4610>, search_kwargs={'k': 3, 'include_metadata': True}), VectorStoreRetriever(tags=['Chroma', 'HuggingFaceBgeEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x7e3e24d80640>, search_kwargs={'k': 3, 'include_metadata': True})])

## 6. Indicizzazione

In [50]:
question = "come si procede ad un sorpasso?"
for chunks in lotr.get_relevant_documents(question):
    print(chunks.page_content)

Valutare se lo spazio libero sia sufficiente ad effettuare correttamente la mano-
vra; accertare che la differenza di velocità tra il nostro veicolo e quella del veico-
lo da superare sia sufficiente; accertare, dal retrovisore, che nessun veicolo che
segue abbia iniziato la manovra di sorpasso; effettuare la manovra con partico-
lare attenzione, data la limitata visibilità notturna. È sba gliato dire che biso gna:
effettuare la manovra anche con scarsa visibilità purché la lar ghezza della strada
lo consenta; accendere in o gni caso anche i proiettori fendinebbia; sor passare
sulla scia (dietro ) di un veicolo che limita la visibilità; rinunciare a sor passare se
il veicolo che precede non ha l'impianto di illuminazione efficiente. 
 
IL SORPASSO È VIETATO: 
Quando ci sia l'apposito segnale; quando non si dispone di uno spazio libero suf-
ficiente; nelle curve e nel tratto ascendente (in salita) del dosso, solo se la stra-
2Indice Argomenti
Libro
Concetti Base
Hardware
Software
M

In [51]:
docs = lotr.get_relevant_documents(question)

## 7. Long Context Reorder

In [52]:
reordering = LongContextReorder()
reordered_docs = reordering.transform_documents(docs)

for chunks in reordered_docs:
    print(chunks.page_content)

2Indice Argomenti
Libro
Concetti Base
Hardware
Software
Misurare la grandezza
Dispositivi di MemoriaInterazione tra 
dispositivi di memoria
Misurare la velocità 
(data rate)
Prestazioni dei 
dispositivi di memoria
Prestazioni del computer
3Libro di riferimento
Ecdl 5.0 Open Source
F. Lunghezzani
D. Princivalle
HOEPLI Informatica
54SOFTWARE
Applicazione : programma (o insieme di programmi) che 
può essere eseguito da un determinato Sistema Operativo
Esempi : elaborazione testi, gestione paghe, fatturazione, 
presentazioni, grafica pittorica e CAD; calcoli scientifici; 
transazioni commerciali
dentemente imposto (non indica la fine del divieto di sorpasso tra le sole auto-
vetture); indica il punto in cui termina il divieto di sorpasso; non consente il sor-
passo se deve essere oltre passata la striscia continua; può essere posto sia nei
centri abitati che fuori e non im pone un particolare limite di velocità. È sba gliato
dire che il se gnale indica la fine del divieto di tr

## 8. Rerank

In [53]:
from flashrank.Ranker import Ranker, RerankRequest
# Small (~34MB), slightly slower & best performance (ranking precision).
ranker = Ranker(model_name="ms-marco-MultiBERT-L-12", cache_dir="/opt")

In [54]:
# Conversione dell'oggetto Document nel formato adatto a flashrank.
c_docs = []
sid = 1
for doc in docs:
  c_docs.append({
      "id": sid,
      "text": doc.page_content,
      "meta": doc.metadata
  })

  sid = sid + 1

In [55]:
rerankrequest = RerankRequest(query=question, passages=c_docs)
results = ranker.rerank(rerankrequest)

for result in results:
  print(result['text'] + "["+str(result['score'])+"]" + "\n")

Manuale di teoria per le patenti A1 A e B 77
Lezione 15. Sorpasso 
 
Questa lezione tratta del sorpasso , chiarendo che cosa fare prima  di iniziare la mano-
vra, durante il rientro  e come comportarsi quando si è sorpassati . 
 
Nella seconda parte si elencheranno i casi in cui il sorpasso è vietato  e quelli in cui è 
consentito solo in particolari condizioni  (da destra, presso un incrocio, in prossimità 
di un dosso). 
 
 
SORPASSO 
Prima di iniziare un sor passo è necessario mantenere la distanza di sicurezza,
accertare che la visibilità sia tale da poterlo fare senza pericolo e che non vi sia-
no segnali che lo vietino; che si disponga di spazio libero sufficiente e che i vei-
coli che se guono o precedono non abbiano iniziato la stessa manovra. Quindi si
deve: segnalare la manovra ai conducenti che seguono azionando l'indicatore di 
direzione sinistro e avvertire quelli che precedono con il clacson (ove consentito )[0.9668422]

Valutare se lo spazio libero sia sufficiente ad eff

## Conclusioni

Il Merge Retriever risulta sicuramente uno strumento molto comodo.
Per quanto concerne invece il Long Context Reorder, rimango perplesso guardando ai risultati ottenuti (almenpo per la lingua italiana).
Sono piacevolmente sorpreso invece dall'efficacia dimostrata dal Reranker Flashrank [https://github.com/PrithivirajDamodaran/FlashRank]